### System setup: object instantiation 

In [1]:
from numpy import zeros, pi, sin, cos, empty, array, float64
#original 
from nvtg2_lj import LJ
from nvtg_write import *
from time import process_time
from  pickle import dump, load, HIGHEST_PROTOCOL

# from numba import jit
if __name__ == "__main__":
    dir="./RUN/"
# r-parameters # defaults
    rho  = 0.984   #  density of the solid phase
    mx   = 16#6       #  number of cells along x
    my   = 16#6 + 2    #  number of cells along y
    mz   = 16 #12+ 2   #  number of cells along z
    kt   = 0.7     #  temperature
    dk0  = 0.00    #  delta T at equilibrium
    dkt  = 0.70    #  delta T for melting region
    dt   = 0.005   #  timestep (LJ units)
    gforce = -0.1  #  vaule of gravity field
    freq =  2      #  printing frequency
    lther=  3      #  width of thermostat region
    nstep= 10      #  number of multiples of 'freq' time steps 
    pfile = dir + "fccmd.pickle"
    print( "# number of boxes mx = %d, my = %d, mz = %d" % (mx, my, mz) )
    print( "# mean (kinetic) temperature kt = %8.4f " % (kt) )
    print( "# integration time step dt = %8.4f" % dt )
    print( "# number of time steps for this run nstep = %d" % (nstep) )    
    t0 = process_time()
    IN = LJ(rho, mx, my, mz, gforce, nstep*freq)
    print("# creating EQ object, cpu = %8.4f" % (process_time()-t0) )
    N = IN.npart
    Na = 3*N // 25
    rhofact = IN.lb / (IN.Lx * IN.Ly * IN.Lz)
    print( "# sides of the rectangular MD-box L = [ %8.4f %8.4f %8.4f ]" % (IN.Lx, IN.Ly, IN.Lz) )
    print( "# number of particles  N = %d" % N)
    print( "# density rho = %8.4f   (%8.4f)" % (IN.rho, rho))
    print( "# potential cut-off radius  rcut = %8.4f *sigma" % IN.rcut)
    print( "# lateral size of slicing Deltaz = %8.4f" % (IN.Lz / IN.lb))

# number of boxes mx = 16, my = 16, mz = 16
# mean (kinetic) temperature kt =   0.7000 
# integration time step dt =   0.0050
# number of time steps for this run nstep = 10
# creating EQ object, cpu =   1.7359
# sides of the rectangular MD-box L = [  51.0707  51.0707  51.0707 ]
# number of particles  N = 114688
# density rho =   0.9840   (  0.9840)
# potential cut-off radius  rcut =   3.0000 *sigma
# lateral size of slicing Deltaz =   1.5960


In [2]:
    # starting from (fcc) lattice
    IN.fcc()
    pas = 0
    #write_input(IN, N, pas, conf_out=dir+"conf_fcc0984.b")
    writexyz(IN, N, Na)

# lattice lenghts (ax,ay,az) = (1.5959586353904496, 1.5959586353904496, 1.5959586353904496)
# (mx, my, mz) = (32, 28, 32)
# number of lattice cells = 28672
# number of particles = 114688
# md-box sides [Lx, Ly, Lz ]= (51.070676332494386, 51.070676332494386, 51.070676332494386)
# end of initial fcc lattice construction npart = 114688


In [4]:
    mode=2
    #pas = read_input(IN, N, conf_in=dir+"conf_fcc0984.b")
    t0 = process_time()    
    (enep, enek, virial, vcmx, vcmy, vcmz) = IN.eqmdi( N, Na, mx, my, mz, kt, pas, mode)
    print( "# initial conditions step = %d, cpu = %8.4f" % (pas, process_time()-t0) )

test 1 -869226.7015878024 -1582637.9261371403
test 2 -271882.6634644713 -8543.583388327661
# velocities sampled from maxwell distribution at timestep 0
# initial conditions step = 0, cpu =   0.6024


In [5]:
    write_input(IN, N, nstep, conf_out=dir+"conf_fcc0984.b")

In [7]:
    nstep= 10      #  number of multiples of 'freq' time steps 
    # Equilibrium run with temperature from maxwellian sampling
    print("# starting eqmd trajectory\n")
    print( "    'pas'   'enep'     'enek'   'enet'      'virial'   'vcm' ")
    print(" %9.2f %9.4f %9.4f %12.7f %8.3f %12.2g %7.2g %7.2g" % (pas*dt,enep, enek, enep+enek, virial, vcmx, vcmy, vcmz) )
    t0 = process_time()
    for it in range(nstep):
        (t, enep, enek, virial, vcmx, vcmy, vcmz) = IN.eqmdr( N, Na, mx, my, mz, kt, pas, dt, freq)
        print(" %9.2f %9.4f %9.4f %12.7f %8.3f %12.2g %7.2g %7.2g" % (t,enep, enek, enep+enek, virial, vcmx, vcmy, vcmz) )
        pas += freq    
    print( "# (IN)-Equilibrium run, cpu = %8.4f" % (process_time()-t0) )
    g=3*N-3
    nstep *= freq
    print( "# ending initial equilibrium run  <ep>=%6.1f  <ek>=%7.4f   T=%8.3f  P=%8.3f\n" % ( IN.ept/nstep, IN.ekt/nstep, 2.*IN.ekt/(nstep*g), IN.pres/nstep) )
    # final positions and momenta to file conf_in.b
    write_input(IN, N, nstep, conf_out=dir+"conf_eg0984.b")
    writexyz(IN, N, Na)

# starting eqmd trajectory

    'pas'   'enep'     'enek'   'enet'      'virial'   'vcm' 
      0.10   -6.8751    0.3410   -6.5341030   -1.317      5.8e-12  -1e+03 -1.4e-12
      0.11   -6.9208    0.3863   -6.5344295   -2.064      5.5e-12 -1.1e+03 -2.1e-12
      0.12   -6.9633    0.4288   -6.5344936   -2.787      9.1e-12 -1.3e+03 1.6e-12
      0.13   -6.9951    0.4607   -6.5343642   -3.347      7.8e-12 -1.4e+03 -2.5e-12
      0.14   -7.0142    0.4801   -6.5341222   -3.701      6.3e-12 -1.5e+03 1.6e-12
      0.15   -7.0225    0.4887   -6.5338371   -3.868      8.8e-12 -1.6e+03 9.8e-13
      0.16   -7.0238    0.4902   -6.5335595   -3.903      5.1e-12 -1.7e+03 -3.2e-13
      0.17   -7.0220    0.4887   -6.5333203   -3.873      4.9e-12 -1.8e+03 -1.9e-13
      0.18   -7.0207    0.4876   -6.5331337   -3.837      7.8e-12 -1.9e+03 -4.4e-12
      0.19   -7.0221    0.4891   -6.5330017   -3.836      9.9e-12  -2e+03 3.3e-13
      0.20   -7.0272    0.4942   -6.5329180   -3.895      5.9e-12 -2.1e+03 -

In [ ]:
    freq = 20
    nstep= 10       #  number of multiples of 'freq' time steps 
    mode = 2
    print("check gforce=",IN.gforce)
    t0 = process_time()    
    pas = read_input(IN, N, conf_in=dir+"conf_eg0984.b")
    (enep, enek, virial, vcmx, vcmy, vcmz) = IN.eqmdi( N, Na, mx, my, mz, kt, pas, mode)
    print( "# initial conditions step = %d, cpu = %8.4f" % (pas, process_time()-t0) )    
    # Equilibrium run with temperature from maxwellian sampling
    print("# starting eqmd trajectory\n")
    print( "    'pas'   'enep'     'enek'   'enet'      'virial'   'vcm' ")
    print("%8.2f  %9.4f %9.4f %12.7f %8.3f %12.2g %7.2g %7.2g" % (pas*dt,enep, enek, enep+enek, virial, vcmx, vcmy, vcmz) )
    t0 = process_time()
    for it in range(nstep):
        (t, enep, enek, virial, vcmx, vcmy, vcmz) = IN.eqmdr( N, Na, mx, my, mz, kt, pas, dt, freq)
        print("%8.2f  %9.4f %9.4f %12.7f %8.3f %12.2g %7.2g %7.2g" % (t,enep, enek, enep+enek, virial, vcmx, vcmy, vcmz) )
        pas += freq    
    print( "# (IN)-Equilibrium run, cpu = %8.4f" % (process_time()-t0) )
    g=3*N-3
    nstep *= freq
    print( "# ending initial equilibrium run  <ep>=%6.1f  <ek>=%7.4f   T=%8.3f  P=%8.3f\n" % ( IN.ept/nstep, IN.ekt/nstep, 2.*IN.ekt/(nstep*g), IN.pres/nstep) )
    # final positions and momenta to file conf_in.b
    write_input(IN, N, nstep, conf_out=dir+"conf_eg0984.b")
    writexyz(IN, N, Na)

In [ ]:
    freq = 20
    nstep= 50      #  number of multiples of 'freq' time steps 
    mode = 1
    # Xe 1.115
    pfile = dir + "eqmd.pickle"
    print( "# number of boxes mx = %d, my = %d, mz = %d" % (mx, my, mz) )
    print( "# mean (kinetic) temperature kt = %8.4f  delta kt= %8.4f" % (kt, dkt) )
    print( "# integration time step dt = %8.4f" % dt )
    print( "# number of time steps for this run nst = %d " % (nstep) )    
    if mode == 1:
        t0 = process_time()
        EQ = LJ(rho, mx, my, mz, gforce, nstep*freq)
        print("# creating NE object, cpu = %8.4f" % (process_time()-t0) )
        N = EQ.npart
        Na = 3*N // 25
        print( "# sides of the rectangular MD-box L = [ %8.4f %8.4f %8.4f ]" % (EQ.Lx, EQ.Ly, EQ.Lz) )
        print( "# number of particles  N = %d" % N)
        print( "# density rho = %8.4f   (%8.4f)" % (EQ.rho, rho) )
        print( "# potential cut-off radius  rcut = %8.4f *sigma" % EQ.rcut)
        print( "# lateral size of slicing Deltaz = %8.4f" % (EQ.Lz / EQ.lb))
        # initial equilibrated condition
        t0 = process_time()
        estep = read_input(EQ, N, conf_in=dir+"conf_eg0984.b")
        print("# initial conditions, cpu = %8.4f" % (process_time()-t0) )
        # Non-Equilibrium run with temperature gradient
        t0 = process_time()
        pas= 0
        (enep, enek, etot, heat0, heat1, vcmx, vcmy, vcmz) = EQ.nemdi( N, Na, mx, my, mz, kt, dk0, pas, lther)
        print("# starting dnemd trajectory\n")
        print( "    'pas'   'enep'     'enek'   'enet'      'heatin - heatout'   'vcm' ")
        print("%8.2f  %9.4f %9.4f %12.7f %8.3f %8.3f %12.2g %7.2g %7.2g" % (0.,enep, enek, etot, heat0, heat1, vcmx, vcmy, vcmz) )
        for it in range(nstep):
            (t, enep, enek, etot, heat0, heat1, vcmx, vcmy, vcmz) = EQ.nemdr( N, Na, mx, my, mz, kt, dk0, pas, dt, freq, lther)
            print("%8.2f  %9.4f %9.4f %12.7f %8.3f %8.3f %12.2g %7.2g %7.2g" % (t,enep, enek, etot, heat0, heat1, vcmx, vcmy, vcmz) )
            pas += freq    
        print( "# (NE)-Equilibrium run, cpu = %8.4f" % (process_time()-t0) )
        g=3*N-3
        nstep *= freq
        print( "# ending ne-md equilibrium trajectory  <ep>=%6.1f  <ek>=%7.4f   T=%8.3f  P=%8.3f\n" % ( EQ.ept/nstep, EQ.ekt/nstep, 2.*EQ.ekt/(nstep*g), EQ.pres/nstep) )
    # final positions and momenta to file conf_in.b
        write_input(EQ, N, nstep, conf_out=dir+"conf_eg0984.b")
        # saving with pickle
        with open(pfile, 'wb') as ftrj:
            rhofact = EQ.lb / (EQ.Lx * EQ.Ly * EQ.Lz)
            dump( (dkt,rhofact,id) , ftrj, HIGHEST_PROTOCOL)
            dump( (EQ.heatt,EQ.hstat,EQ.hstbt,EQ.enet,EQ.enkat,EQ.enkbt,EQ.jmazt,EQ.gzt,EQ.jezt), ftrj, HIGHEST_PROTOCOL)

In [ ]:
    writexyz(IN, N, Na)

## Non-equilibrium dynamics

In [ ]:
from numpy import zeros, pi, sin, cos, empty, array, float64
from nvtg2_lj import LJ
from nvtg_write import *
from time import process_time
from  pickle import dump, load, HIGHEST_PROTOCOL

# from numba import jit
if __name__ == "__main__":
    dir="./RUN/"
    pfile = dir + "ngmdA.pickle"
# r-parameters     # defaults
    nstep= 200     #  number of multiples of 'freq' time steps 
    rho  = 0.984   #  density of the solid phase
    mx   = 6       #  number of cells along x
    my   = 6 + 2   #  number of cells along y
    mz   = 12+ 2   #  number of cells along z
    kt   = 0.7     #  temperature
    dk0  = 0.00    #  delta T at equilibrium
    dkt  = 0.70    #  delta T for melting region
    dt   = 0.005   #  timestep (LJ units)
    gforce = -0.1  #  vaule of gravity field
    freq = 10      #  printing frequency
    lther=  3      #  width of thermostat region
    mode = 1
    print( "# number of boxes mx = %d, my = %d, mz = %d" % (mx, my, mz) )
    print( "# base (kinetic) temperature kt = %8.4f  delta kt= %8.4f" % (kt, dkt) )
    print( "# integration time step dt = %8.4f" % dt )
    print( "# number of time steps for this run nst = %d " % (nstep) )    
    with open(dir+"trajectoryAg.xyz", 'w') as dumpf:
        t0 = process_time()
        NE = LJ(rho, mx, my, mz, gforce, nstep*freq)
        print("# creating NE object, cpu = %8.4f" % (process_time()-t0) )
        N = NE.npart
        Na = 3*N // 25
        rhofact = NE.lb / (NE.Lx * NE.Ly * NE.Lz)
        print( "# sides of the rectangular MD-box L = [ %8.4f %8.4f %8.4f ]" % (NE.Lx, NE.Ly, NE.Lz) )
        print( "# number of particles  N = %d" % N)
        print( "# density rho = %8.4f   (%8.4f)" % (NE.rho, rho) )
        print( "# potential cut-off radius  rcut = %8.4f *sigma" % NE.rcut)
        print( "# lateral size of slicing Deltaz = %8.4f" % (NE.Lz / NE.lb))
        # initial equilibrated condition
        t0 = process_time()
        estep = read_input(NE, N, conf_in=dir+"conf_eg0984.b")
        print("# initial conditions, cpu = %8.4f" % (process_time()-t0) )
        # Non-Equilibrium run with hot thermostat region
        t0 = process_time()
        pas= 0
        dumpxyz(NE, N, Na, pas, dumpf)
        (enep, enek, etot, heat0, heat1, vcmx, vcmy, vcmz) = NE.nemdi( N, Na, mx, my, mz, kt, dkt, pas, lther )
        print("# starting dnemd trajectory\n")
        print( "    'pas'   'enep'     'enek'   'enet'      'heatin - heatout'   'vcm' ")
        print("%8.2f  %9.4f %9.4f %12.7f %8.3f %8.3f %12.2g %7.2g %7.2g" % (0.,enep, enek, etot, heat0, heat1, vcmx, vcmy, vcmz) )
        for it in range(nstep):
            (t, enep, enek, etot, heat0, heat1, vcmx, vcmy, vcmz) = NE.nemdr( N, Na, mx, my, mz, kt, dkt, pas, dt, freq, lther )
            print("%8.2f  %9.4f %9.4f %12.7f %8.3f %8.3f %12.2g %7.2g %7.2g" % (t,enep, enek, etot, heat0, heat1, vcmx, vcmy, vcmz) )
            pas += freq
            # writing 
            dumpxyz(NE, N, Na, pas, dumpf)
    print( "# Non-Equilibrium run, cpu = %8.4f" % (process_time()-t0) )
    g=3*N-3
    nstep *= freq
    print( "# ending ne-md trajectory  <ep>=%6.1f  <ek>=%7.4f   T=%8.3f  P=%8.3f\n" % ( NE.ept/nstep, NE.ekt/nstep, 2.*NE.ekt/(nstep*g), NE.pres/nstep) )
    # final positions and momenta to file conf_in.b
    write_input(NE, N, nstep, conf_out=dir+"conf_ng0984.b")
    # saving with pickle
    with open(pfile, 'wb') as ftrj:
        rhofact = NE.lb / (NE.Lx * NE.Ly * NE.Lz)
        dump( (dkt,rhofact,id) , ftrj, HIGHEST_PROTOCOL)
        dump( (NE.heatt,NE.hstat,NE.hstbt,NE.enet,NE.enkat,NE.enkbt,NE.jmazt,NE.gzt,NE.jezt), ftrj, HIGHEST_PROTOCOL)


### second part of trajectory

In [ ]:
if __name__ == "__main__":
    dir="./RUN/"
    pfile = dir + "ngmdB.pickle"
# r-parameters  # defaults
    nstep= 1000  #  number of multiples of 'freq' time steps 
    freq = 200  #  2000
    print( "# number of boxes mx = %d, my = %d, mz = %d" % (mx, my, mz) )
    print( "# base (kinetic) temperature kt = %8.4f  delta kt= %8.4f" % (kt, dkt) )
    print( "# integration time step dt = %8.4f" % dt )
    print( "# number of time steps for this run nst = %d " % (nstep) )    
    with open(dir+"trajectoryBg.xyz", 'w') as dumpf:
        t0 = process_time()
        NE = LJ(rho, mx, my, mz, gforce, nstep*freq)
        print("# creating NE object, cpu = %8.4f" % (process_time()-t0) )
        N = NE.npart
        Na = 3*N // 25
        rhofact = NE.lb / (NE.Lx * NE.Ly * NE.Lz)
        print( "# sides of the rectangular MD-box L = [ %8.4f %8.4f %8.4f ]" % (NE.Lx, NE.Ly, NE.Lz) )
        print( "# number of particles  N = %d" % N)
        print( "# density rho = %8.4f   (%8.4f)" % (NE.rho, rho) )
        print( "# potential cut-off radius  rcut = %8.4f *sigma" % NE.rcut)
        print( "# lateral size of slicing Deltaz = %8.4f" % (NE.Lz / NE.lb))
        # initial equilibrated condition
        t0 = process_time()
        estep = read_input(NE, N, conf_in=dir+"conf_ng0984.b")
        print("# initial conditions, cpu = %8.4f" % (process_time()-t0) )
        # Non-Equilibrium run with temperature gradient
        t0 = process_time()
        pas= 0
        dumpxyz(NE, N, Na, pas, dumpf)
        (enep, enek, etot, heat0, heat1, vcmx, vcmy, vcmz) = NE.nemdi( N, Na, mx, my, mz, kt, dkt, pas, lther )
        print("# starting dnemd trajectory\n")
        print( "    'pas'   'enep'     'enek'   'enet'      'heatin - heatout'   'vcm' ")
        print(" %10.1f %9.4f %9.4f %12.7f %8.3f %8.3f %12.2g %7.2g %7.2g" % (0.,enep, enek, etot, heat0, heat1, vcmx, vcmy, vcmz) )
        for it in range(nstep):
            (t, enep, enek, etot, heat0, heat1, vcmx, vcmy, vcmz) = NE.nemdr( N, Na, mx, my, mz, kt, dkt, pas, dt, freq, lther )
            print(" %10.1f %9.4f %9.4f %12.7f %8.3f %8.3f %12.2g %7.2g %7.2g" % (t,enep, enek, etot, heat0, heat1, vcmx, vcmy, vcmz) )
            pas += freq
            # writing 
            dumpxyz(NE, N, Na, pas, dumpf)
    print( "# Non-Equilibrium run, cpu = %8.4f" % (process_time()-t0) )
    g=3*N-3
    nstep *= freq
    print( "# ending ne-md trajectory  <ep>=%6.1f  <ek>=%7.4f   T=%8.3f  P=%8.3f\n" % ( NE.ept/nstep, NE.ekt/nstep, 2.*NE.ekt/(nstep*g), NE.pres/nstep) )
    # final positions and momenta to file conf_in.b
    write_input(NE, N, nstep, conf_out=dir+"conf_ng0984.b")
    # saving with pickle
    with open(pfile, 'wb') as ftrj:
        rhofact = NE.lb / (NE.Lx * NE.Ly * NE.Lz)
        dump( (dkt,rhofact,id) , ftrj, HIGHEST_PROTOCOL)
        dump( (NE.heatt,NE.hstat,NE.hstbt,NE.enet,NE.enkat,NE.enkbt,NE.jmazt,NE.gzt,NE.jezt), ftrj, HIGHEST_PROTOCOL)
